# Music Industry Sales Kaggle Project

## Introduction

### Main Stakeholders

Because this dataset is about music sales based on different formats (CDs, cassette, vinyls, etc.), the main stakeholder will be the record label company. The company will ultimately want to know: what's the best (most profitable) format to release their artists' songs to?


### Business Tasks

To help guide my analysis, I want to answer these main questions:

- How many units were sold per song format?
- How much money did each format make in total?
- Adjusting for inflation, how much money is that today and which format was the most profitable?
- Are there any discrepencies or unusual trends (eg. random peak of a certain format)?
- What's the best (most profitable) format for new song releases?

In [78]:
# Imports
import numpy as np
import pandas as pd

## Exploratory Data Analysis (EDA)

In [79]:
# Read csv file
df = pd.read_csv('musicdata.csv')
df

,format,metric,year,number_of_records,value_actual
0,CD,Units,1973,1,NaN
1,CD,Units,1974,1,NaN
2,CD,Units,1975,1,NaN
3,CD,Units,1976,1,NaN
4,CD,Units,1977,1,NaN
...,...,...,...,...,...
3003,Vinyl Single,Value (Adjusted),2015,1,6.205390
3004,Vinyl Single,Value (Adjusted),2016,1,5.198931
3005,Vinyl Single,Value (Adjusted),2017,1,6.339678
3006,Vinyl Single,Value (Adjusted),2018,1,5.386197


### Introductory Look

In [80]:
df.head()

,format,metric,year,number_of_records,value_actual
0,CD,Units,1973,1,NaN
1,CD,Units,1974,1,NaN
2,CD,Units,1975,1,NaN
3,CD,Units,1976,1,NaN
4,CD,Units,1977,1,NaN


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   format             3008 non-null   object 
 1   metric             3008 non-null   object 
 2   year               3008 non-null   int64  
 3   number_of_records  3008 non-null   int64  
 4   value_actual       1351 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 117.6+ KB


In [82]:
df.describe()

,year,number_of_records,value_actual
count,3008.000000,3008.0,1351.000000
mean,1996.000000,1.0,781.291237
std,13.566915,0.0,2246.837672
min,1973.000000,1.0,-7.650944
25%,1984.000000,1.0,3.700228
50%,1996.000000,1.0,63.900000
75%,2008.000000,1.0,448.900000
max,2019.000000,1.0,19667.327786


In [83]:
df['format'].value_counts()

CD                                    141
DVD Audio                             141
Ringtones & Ringbacks                 141
Download Music Video                  141
Kiosk                                 141
CD Single                             141
Download Single                       141
SACD                                  141
Download Album                        141
Music Video (Physical)                141
Other Tapes                           141
8 - Track                             141
Vinyl Single                          141
LP/EP                                 141
Cassette Single                       141
Cassette                              141
Paid Subscriptions                     94
Limited Tier Paid Subscription         94
On-Demand Streaming (Ad-Supported)     94
Other Ad-Supported Streaming           94
Other Digital                          94
Paid Subscription                      94
SoundExchange Distributions            94
Synchronization                   

It looks like there's a lot fewer values from subscriptions, streaming, and other digital services. Also is there any difference between 'Paid Subscriptions' and 'Paid Subscription'?

In [84]:
df['metric'].value_counts()

Value               1081
Value (Adjusted)    1081
Units                846
Name: metric, dtype: int64

So from an introductory look of the data, there are:

- 3 numerical columns: 'year', 'number_of_records', 'value_actual'
- 2 text columns: 'format', 'metric'

Since 'format' and 'metric' are both object dtypes, they need to be converted to string dtype.

In [85]:
df['number_of_records'].value_counts()

1    3008
Name: number_of_records, dtype: int64

The 'number_of_records' column description states: "Unit (all rows are 1)." I assume this means that each observation (row) is 1 sale (eg. 1 cd sale, 1 cassette sale, 1 paid subscription sale, etc). I think it is safe enough to drop as it doesn't provide any useful information as each observation already denotes 1 sale.

In [86]:
# Null values ratio
num_null_values = df['value_actual'].isnull().groupby(df['format']).sum()
num_null_values / df['format'].value_counts()

8 - Track                             0.581560
CD                                    0.212766
CD Single                             0.319149
Cassette                              0.205674
Cassette Single                       0.595745
DVD Audio                             0.595745
Download Album                        0.659574
Download Music Video                  0.680851
Download Single                       0.659574
Kiosk                                 0.680851
LP/EP                                 0.000000
Limited Tier Paid Subscription        0.914894
Music Video (Physical)                0.340426
On-Demand Streaming (Ad-Supported)    0.808511
Other Ad-Supported Streaming          0.914894
Other Digital                         0.914894
Other Tapes                           0.666667
Paid Subscription                     0.680851
Paid Subscriptions                    0.691489
Ringtones & Ringbacks                 0.680851
SACD                                  0.638298
SoundExchange

It looks like there's quite a lot of missing values. A couple formats such as 'Limited Tier Paid Subscription' and 'Other Ad-Supported Streaming' have over 91% missing values, while the majority sit around 60%-70%.

Only 'LP/EP' and 'Vinyl Single' formats do not have any missing values.

Let's take a look why that is.

In [87]:
df.loc[(df['format'] == 'Limited Tier Paid Subscription') & (df['value_actual'].isnull())].describe()

,year,number_of_records,value_actual
count,86.000000,86.0,0.0
mean,1994.000000,1.0,NaN
std,12.482458,0.0,NaN
min,1973.000000,1.0,NaN
25%,1983.250000,1.0,NaN
50%,1994.000000,1.0,NaN
75%,2004.750000,1.0,NaN
max,2015.000000,1.0,NaN


In [88]:
df.loc[(df['format'] == 'Limited Tier Paid Subscription') & (df['value_actual'].notnull())].describe()

,year,number_of_records,value_actual
count,8.000000,8.0,8.000000
mean,2017.500000,1.0,614.897193
std,1.195229,0.0,228.885536
min,2016.000000,1.0,263.363685
25%,2016.750000,1.0,513.834996
50%,2017.500000,1.0,682.043612
75%,2018.250000,1.0,777.818209
max,2019.000000,1.0,829.498740


In [89]:
df.loc[(df['format'] == 'Other Digital') & (df['value_actual'].isnull())].describe()

,year,number_of_records,value_actual
count,86.000000,86.0,0.0
mean,1994.000000,1.0,NaN
std,12.482458,0.0,NaN
min,1973.000000,1.0,NaN
25%,1983.250000,1.0,NaN
50%,1994.000000,1.0,NaN
75%,2004.750000,1.0,NaN
max,2015.000000,1.0,NaN


In [90]:
df.loc[(df['format'] == 'Other Digital') & (df['value_actual'].notnull())].describe()

,year,number_of_records,value_actual
count,8.000000,8.0,8.000000
mean,2017.500000,1.0,19.121265
std,1.195229,0.0,1.893653
min,2016.000000,1.0,16.885110
25%,2016.750000,1.0,17.493630
50%,2017.500000,1.0,19.049492
75%,2018.250000,1.0,20.528591
max,2019.000000,1.0,21.517175


In [91]:
df.loc[(df['format'] == 'DVD Audio') & (df['value_actual'].isnull())].describe()

,year,number_of_records,value_actual
count,84.000000,84.0,0.0
mean,1986.500000,1.0,NaN
std,8.126263,0.0,NaN
min,1973.000000,1.0,NaN
25%,1979.750000,1.0,NaN
50%,1986.500000,1.0,NaN
75%,1993.250000,1.0,NaN
max,2000.000000,1.0,NaN


In [92]:
df.loc[(df['format'] == 'DVD Audio') & (df['value_actual'].notnull())].describe()

,year,number_of_records,value_actual
count,57.000000,57.0,57.000000
mean,2010.000000,1.0,2.699240
std,5.525913,0.0,3.632631
min,2001.000000,1.0,-0.544117
25%,2005.000000,1.0,0.179507
50%,2010.000000,1.0,1.067303
75%,2015.000000,1.0,3.317384
max,2019.000000,1.0,14.661333


In [93]:
df.loc[(df['format'] == 'CD Single') & (df['value_actual'].isnull())].describe()

,year,number_of_records,value_actual
count,45.000000,45.0,0.0
mean,1980.000000,1.0,NaN
std,4.369314,0.0,NaN
min,1973.000000,1.0,NaN
25%,1976.000000,1.0,NaN
50%,1980.000000,1.0,NaN
75%,1984.000000,1.0,NaN
max,1987.000000,1.0,NaN


In [94]:
df.loc[(df['format'] == 'CD Single') & (df['value_actual'].notnull())].describe()


,year,number_of_records,value_actual
count,96.000000,96.0,96.000000
mean,2003.500000,1.0,45.346843
std,9.281561,0.0,85.966719
min,1988.000000,1.0,-1.443225
25%,1995.750000,1.0,1.434021
50%,2003.500000,1.0,6.650000
75%,2011.250000,1.0,45.275000
max,2019.000000,1.0,434.377968


Looking at the min and max year to find the range,

null values are present during:

- Limited Tier Paid Subscription: 1973-2015
- Other Digital: 1973-2015
- DVD Audio: 1973-2000
- CD Single: 1973-1987

non-null values are present during:

- Limited Tier Paid Subscription: 2016-2019
- Other Digital: 2016-2019
- DVD Audio: 2001-2019
- CD Single: 1988-2019

The null values correspond to the years for which the specific format hasn't been introduced yet or has been discontinued (eg. Limited Tier Paid Subscription didn't start until 2016 so all years before 2016 are null values). This makes a lot of sense because formats follow the rise and fall of technologies - cassettes were replaced by CDs, which were replaced by DVDs, which were replaced by digital formats such as mp3 and subscriptions.

Now let's take a look into the 'Paid Subscriptions' vs 'Paid Subscription'.

In [95]:
df.loc[df['format'] == 'Paid Subscriptions'].describe(include='all')

,format,metric,year,number_of_records,value_actual
count,94,94,94.000000,94.0,29.000000
unique,1,1,NaN,NaN,NaN
top,Paid Subscriptions,Units,NaN,NaN,NaN
freq,94,94,NaN,NaN,NaN
mean,NaN,NaN,1996.000000,1.0,12.118509
std,NaN,NaN,13.637393,0.0,17.023962
min,NaN,NaN,1973.000000,1.0,1.200000
25%,NaN,NaN,1984.250000,1.0,1.600000
50%,NaN,NaN,1996.000000,1.0,3.360000
75%,NaN,NaN,2007.750000,1.0,10.840000


In [96]:
df.loc[df['format'] == 'Paid Subscription'].describe(include='all')

,format,metric,year,number_of_records,value_actual
count,94,94,94.000000,94.0,30.000000
unique,1,2,NaN,NaN,NaN
top,Paid Subscription,Value,NaN,NaN,NaN
freq,94,47,NaN,NaN,NaN
mean,NaN,NaN,1996.000000,1.0,1417.139072
std,NaN,NaN,13.637393,0.0,1827.921687
min,NaN,NaN,1973.000000,1.0,149.200000
25%,NaN,NaN,1984.250000,1.0,246.241544
50%,NaN,NaN,1996.000000,1.0,422.579419
75%,NaN,NaN,2007.750000,1.0,1995.034908


It looks like these 2 columns are describing the same 'Paid Subscription' but are using different metrics, 'Paid Subscriptions' has metric in units and 'Paid Subscription' has metric of value and value (adjusted). I'll combine these 2 columns into 'Paid Subscription'.

Because the dataset did not provide any further explanation on what the metrics mean, I am assuming they are defined as such:

- Units: The quantity sold (eg. 1 CD Single, 2 Cassette)
- Value: The monetary price, in dollars
- Value (Adjusted): The adjusted monetary price to 2019, in dollars

Lastly, I want to look at the few negative numbers in 'value_actual'.

In [97]:
df.loc[df['value_actual'] < 0]

,format,metric,year,number_of_records,value_actual
63,CD Single,Units,1989,1,-0.100000
169,Cassette Single,Units,2001,1,-1.500000
170,Cassette Single,Units,2002,1,-0.500000
463,DVD Audio,Units,2013,1,-0.054735
909,CD Single,Value,1989,1,-0.700000
1062,Cassette Single,Value,2001,1,-5.300000
1063,Cassette Single,Value,2002,1,-1.600000
1121,DVD Audio,Value,2013,1,-0.495804
2037,CD Single,Value (Adjusted),1989,1,-1.443225
2143,Cassette Single,Value (Adjusted),2001,1,-7.650944


The negative numbers are outliers because there is no such thing as a negative unit or price. There are only 12 instances of these, so I will replace these values with 0. For the instance of Cassette Single, the negative numbers appear at the tail end, right before the format is dropped off, which suggests that replacing with a 0 is okay.

## Data Cleaning

Now let's clean the data.

- Convert 'format' and 'metric' columns from object dtype to string dtype
- Drop 'number_of_records' column
- Combine 'Paid Subscriptions' and 'Paid Subscriptions' into 1 value: 'Paid Subscription'
- Replace all null values in 'value_actual' with 0
- Replace all negative values in 'value_actual' with 0

In [98]:
# Convert to string dtype
df['format'] = df['format'].astype('string')
df['metric'] = df['metric'].astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   format             3008 non-null   string 
 1   metric             3008 non-null   string 
 2   year               3008 non-null   int64  
 3   number_of_records  3008 non-null   int64  
 4   value_actual       1351 non-null   float64
dtypes: float64(1), int64(2), string(2)
memory usage: 117.6 KB


In [99]:
# Drop number_of_records column
df.drop('number_of_records', axis=1, inplace=True)
df.columns

Index(['format', 'metric', 'year', 'value_actual'], dtype='object')

In [100]:
# Combine Paid Subscription(s) columns into 1
df['format'].replace('Paid Subscriptions', 'Paid Subscription', inplace=True)
df['format'].value_counts()

Paid Subscription                     188
Download Single                       141
Cassette                              141
Cassette Single                       141
LP/EP                                 141
Vinyl Single                          141
8 - Track                             141
CD                                    141
Music Video (Physical)                141
Other Tapes                           141
SACD                                  141
CD Single                             141
Download Album                        141
Kiosk                                 141
Download Music Video                  141
Ringtones & Ringbacks                 141
DVD Audio                             141
Limited Tier Paid Subscription         94
On-Demand Streaming (Ad-Supported)     94
Other Ad-Supported Streaming           94
Other Digital                          94
SoundExchange Distributions            94
Synchronization                        94
Name: format, dtype: Int64

In [101]:
# Replace all null values in 'value_actual' with 0
df['value_actual'].fillna(0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   format        3008 non-null   string 
 1   metric        3008 non-null   string 
 2   year          3008 non-null   int64  
 3   value_actual  3008 non-null   float64
dtypes: float64(1), int64(1), string(2)
memory usage: 94.1 KB


In [102]:
# Replace all negative values in 'value_actual' with 0
df.loc[df['value_actual'] < 0, 'value_actual'] = 0
df.loc[df['value_actual'] < 0]

,format,metric,year,value_actual


The dataset is now clean and ready to be analyzed.

## Data Analysis